# __Imports__

In [1]:
# [+] Libraries that are commented out are only for ROS
#import rospy
#from std_msgs.msg import Float64
#from sensor_msgs.msg import Image
#from cv_bridge import CvBridge, CvBridgeError
import numpy as np
import cv2
import math
import time
import sys
import argparse
import imutils

# __Gate__

In [5]:
import numpy as np
import cv2
import math
import time
import sys

current_yaw = 0
cv_image = 0
YAW_VARIANCE = .017

# [+]- Access the camera, the int parameter determines which camera you are using, may have to change in depending on what computer you run on
video_capture = cv2.VideoCapture(0)
# video_capture = cv2.VideoCapture(1)
# video_capture = cv2.VideoCapture(2)

#[+]- Each iteration of this loop processes a frame that is captured by the camera by applying a series of filters. Each filter is in intermediate step, the final image is the one which we annotate and extract information from. 
while(True):
    # Capture the raw frame
    ret, frame = video_capture.read()
    final = frame

    # change the frame from bgr values to hsv values
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # [+] specify the lower and upper bounds of colors to be NOT filtered out
    lower_color_bounds = np.array([0, 100, 20])
    upper_color_bounds = np.array([30, 255, 255])

    # [+]- Filter 1: Take the frame, filter out the pixels that are NOT included in the lower and upper bounds, threshold shows in black the pixels being filtered out, and the pixels that are included are white
    threshold = cv2.inRange(hsv, lower_color_bounds, upper_color_bounds)

    # [+]- Filter 2:  Take the thresholded image, and erode to remove noise
    kernel = np.ones((10, 10), np.uint8)
    erode = cv2.erode(threshold, kernel)

    # Get the contours from the eroded image, Each individual contour is a Numpy array of (x,y) coordinates of boundary points of the object
    contours = cv2.findContours(erode, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[0]

    # Get the areas of each contour. Store each contour area in an array, 
    contour_areas = [cv2.contourArea(x) for x in contours]  
    
    # Sort the array of areas from smallest to largest, returns a list of indeces for contour areas array 
    contour_indexes = np.argsort(contour_areas)  
    
    # Sort the indexes of the largest areas
    for i in contour_indexes[-2:]:  # only look at the two largest contours
        (x,y,w,h) = cv2.boundingRect(contours[i])  # get the location/dimensions of a bounding box for the contour: x,y=coordinates, w,h=dims
        cv2.rectangle(final, pt1=(x,y), pt2=(x+w,y+h), color=(255,0,0), thickness=5)  # draw the bounding box on the image

        # for visibility, we will place a background fill on the contour label
        text = "gatepost"
        text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_PLAIN, 1, 1)
        text_w, text_h = text_size
        cv2.rectangle(final, pt1=(x, y), pt2=(x + text_w, y - 2*text_h), color=(255, 0, 0), thickness=-1)
        cv2.putText(final, "gatepost", org=(x, y-5), fontFace=cv2.FONT_HERSHEY_PLAIN, fontScale=1, color=(255, 255, 255), thickness=1)

    # [+]- Show each filter that is applied to the frame. Each filter is shown in a separate window
    cv2.imshow('1 Gate: original frame',frame)
    cv2.imshow('2 Gate: threshold', threshold)
    cv2.imshow("3 Gate: eroded", erode)
    cv2.imshow("4 Gate: final", final)


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
#[+]:
cv2.destroyAllWindows()

error: OpenCV(4.0.1) C:\ci\opencv-suite_1573470242804\work\modules\imgproc\src\color.cpp:181: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


# __Buoy__ 

In [2]:
import cv2
import numpy as np
import math
import time
import sys

current_yaw = 0
cv_image = 0
YAW_VARIANCE = .017

# [+]- Access the camera, camNum determines which camera you are using, may have to change in depending on what computer you run on
camNum= 0
video_capture = cv2.VideoCapture(camNum)


#[+]- Each iteration of this loop processes a frame that is captured by the camera by applying a series of filters. Each filter is in intermediate step, the final image is the one which we annotate and extract information from. 
while(True):
    # Capture the raw frame
    ret, img_frame = video_capture.read()
    img_final = img_frame

    # change the frame from bgr values to hsv values
    img_hsv = cv2.cvtColor(img_frame, cv2.COLOR_BGR2HSV)



#[+]======================================================================================================================================================\\
    # [+] Filter 1: Take the frame, filter out the pixels that are NOT included in the lower and upper bounds, threshold shows in black the pixels being filtered out, and the pixels that are included are white
    lower_value_bounds = np.array([0, 0, 0])
    upper_value_bounds = np.array([255, 255, 45])
    img_threshold = cv2.inRange(img_hsv, lower_value_bounds, upper_value_bounds)

    #[+] Filter 2: Take the threshold image and dilate it to make en thicker
    dilateKernel = np.ones((5, 5), np.uint8)
    img_dilate = cv2.dilate(img_threshold, dilateKernel, iterations=3)

    # [+]- Filter 3:  Take the dilated image, and erode it to remove noise
    erodeKernel = np.ones((10, 10), np.uint8)
    img_erode = cv2.erode(img_dilate, erodeKernel, iterations=2)

    #[+] Filter 4: Take the erode image and blur to to smooth edges so circle detection is easier. At this point, the circle, like the one on the badge, should be extra visible
    blurnel = (5, 5)
    img_blur = cv2.blur(img_erode, ksize= blurnel)

    #[+] Circle detection: Take the blurred image, detect any circles in may have, and store them in this 'circles' array
    circles = cv2.HoughCircles(img_blur, cv2.HOUGH_GRADIENT, 1, 20,
                  param1=30,  # edge detection parameter
                  param2=30,  # accumulator threshold, or how circley the circles need to be to be recognized (higher=more circlely)
                  minRadius=0,
                  maxRadius=100)
#[+]=====================================================================================================================================================//



    # [+] Check if the circles array is not empty, if so, render over every circle in the array of circles:
    if (type(circles)) is np.ndarray:

        #[+] Get the radius of each circle and sort indeces from smallest to largest
        circle_radii = [x[2] for x in circles[0]]  
        circle_indexes = np.argsort(circle_radii)  

        #[+] Render every circle in the circles array. Negative indicies count from the end to the beginning of the array
        for i in circle_indexes[-2:]:  # only contour at the largest circles

            #[+] Get the largest circle, specify the parameters, and render the circle: 
            circle = circles[0][i]  
            cv2.circle(img_final, 
                       center=(int(circle[0]), int(circle[1])), 
                       radius=int(circle[2]), 
                       color=(100, 0, 255), 
                       thickness=2
            )  

            # [+] Create the text, and get its size and dimensions, and determine the cooridnates of its origin (center)
            text = " -->   <-- "
            text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_PLAIN, 1, 1)  
            text_w, text_h = text_size     
            origX= int(circle[0])-text_w
            origY= int(circle[1])+(text_h//2)+8                                     

            #[+] Render the text with its size, dimensions, and origin onto the target frame (final frame)
            cv2.putText(img_final, text, 
                org=(origX, origY ), 
                fontFace=cv2.FONT_HERSHEY_PLAIN, 
                fontScale=2, 
                color=(100, 0, 255), 
                thickness=2
            )


    # [+] Show each step of the image processing pipeline, each window displays the result of a separate filter
    cv2.imshow('1 original frame', img_frame)
    cv2.imshow('2 Buoy: hsv', img_hsv)
    cv2.imshow('3 Buoy: threshold', img_threshold)
    cv2.imshow("4 Buoy: dilated", img_dilate)
    cv2.imshow("5 Buoy: eroded", img_erode)
    cv2.imshow("6 Buoy: blur", img_blur)
    cv2.imshow("7 Buoy: final", img_final)


    #[+] When you hit "q", it exits the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

#[+] When you exit the loop, it closes the program
cv2.destroyAllWindows()


error: OpenCV(4.0.1) C:\ci\opencv-suite_1573470242804\work\modules\imgproc\src\color.cpp:181: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


# Rectangular


In [10]:
import cv2
import numpy as np

def nothing(x):
    pass

cap = cv2.VideoCapture(0)

cv2.namedWindow("Trackbars")
cv2.createTrackBar("Lower H", "Trackbars", 0, 180, nothing)
cv2.createTrackBar("Lower S", "Trackbars", 0, 255, nothing)
cv2.createTrackBar("Lower V", "Trackbars", 0, 255, nothing)
cv2.createTrackBar("Upper H", "Trackbars", 180, 180, nothing)
cv2.createTrackBar("Upper S", "Trackbars", 255, 255, nothing)
cv2.createTrackBar("Upper V", "Trackbars", 255, 255, nothing)



while True:
    _, frame = cap.read()
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)


    lower_red = np.array([0,40,0])
    upper_red = np.array([180, 255, 255])

    mask = cv2.inRange(hsv, lower_red, upper_red)

   # _, contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

   # for cnt in contours:
   #     approx = cv2.approxPolyDP(cnt, 0.01&cv2.arcLength(cnt, True), True)
   #     cv2.drawContours(frame, [cnt], 0, (0, 0, 0), 5)

   #     if len(approx) == 4:
   #         print("It's a rectangle")

    cv2.imshow("Frame", frame)
    cv2.imshow("Mask", mask)


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

#[+] When you exit the loop, it closes the program
cv2.destroyAllWindows()

